In [5]:
from collections import defaultdict
import json
import os
import re
import ipywidgets as widgets
import time
from IPython.display import display,clear_output

In [2]:
def template_arg_fn_sp(example,name_mapping,ctx_args):
    ctx_args['head_item'] = name_placeholder(example['h'],**name_mapping)
    ctx_args['head_mainpart'] = name_placeholder(
        example['h'].removeprefix("<某人X>"),**name_mapping)
    if 't' in example:
        ctx_args['tail_item'] = name_placeholder(example['t'],**name_mapping)
    #BUGGY 先这么实现，有可能有问题，后面再考虑
    ctx_args['X'] = name_mapping.get('<某人X>') or name_mapping.get('<某样东西X>') 
    ctx_args['Y'] = name_mapping.get('<某人Y>') or name_mapping.get('<某样东西Y>')
    ctx_args['Z'] = name_mapping.get('<某人Z>') or name_mapping.get('<某样东西Z>')
    return ctx_args

def name_placeholder(s,**kwargs):
    for k,v in kwargs.items():
        s = s.replace(k,v)
    return s

def scrub_placeholder(s,**kwargs):
    for k,v in kwargs.items():
        s = s.replace(v,k)
    return s

def scrub_placeholder_plus(s,name_mapping,additional_name_pattern,additional_placeholder_list=["<某人Y>","<某人Z>"]):
    #先尝试用name_mapping替换，没有的话就匹配additional_names中的名字，然后从未使用的place_holder_list里找靠前的替换
    used_placeholder = set()
    for k,v in name_mapping.items():
        s,count = re.subn(v,k,s)
        if count:
            used_placeholder.add(v)
    match_names = re.findall(additional_name_pattern,s)
    match_names = set(match_names)
    i = 0
    for match_name in match_names:
        while additional_placeholder_list[i] in used_placeholder:
            i+=1
        s,count = re.subn(match_name,additional_placeholder_list[i],s)
        used_placeholder.add(additional_placeholder_list[i])
    return s

def example2text(example,name_mapping,example_prompt_template,arg_fn,**ctx_args):
    ctx_args = arg_fn(example,name_mapping,ctx_args)
    ctx_args = defaultdict(str,ctx_args)
    example_prompt_text = example_prompt_template.format_map(ctx_args)
    return example_prompt_text

def get_prompt(examples,generating_instance,name_mapping={},add_number=True,separator="；",
    prefix_prompt_text="",example_prompt_template="",generate_prompt_template="",
    arg_fn=template_arg_fn_sp
    ):
    prompt_text = prefix_prompt_text
    for i,example in enumerate(examples):
        if isinstance(name_mapping,list):
            nm = name_mapping[i]
        else:
            nm = name_mapping
        if add_number:
            ctx = {"idx":i+1}
        else:
            ctx = {}
        example_text = example2text(
            example,name_mapping=nm,
            example_prompt_template=example_prompt_template,arg_fn=arg_fn,
            **ctx
        )
        prompt_text+=example_text + separator
    i+=1
    if isinstance(name_mapping,list):
        nm = name_mapping[i]
    else:
        nm = name_mapping
    if add_number:
        ctx = {"idx":i+1}
    else:
        ctx = {} 
    prompt_text+= example2text(
        generating_instance,name_mapping=nm,
        example_prompt_template=generate_prompt_template,arg_fn=arg_fn,
        **ctx
    )
    return prompt_text

In [3]:
workspace = "./"
with open(os.path.join(workspace,"annotator.txt")) as f:
    annotator = f.readline().strip()
if os.path.exists((os.path.join(workspace,"annotation_states.json"))):
    with open(os.path.join(workspace,"annotation_states.json")) as f:
        annotation_states = json.load(f)
        farthest_annotated_idx = annotation_states['farthest_annotated_idx']
        current_idx = farthest_annotated_idx+1
        # annotation_choices = annotation_states['annotation_choices']
        # enable_auto_filter = annotation_states['enable_auto_filter']
        # auto_filter_mappings = annotation_states["auto_filter_mappings"]
 
else:
    # annotation_choices = [
    #     "总是成立或经常成立",
    #     "有时成立或倾向成立",
    #     "很难成立或从不成立",
    #     "无意义、无效表达"
    #     # "头无意义", #有句法和表达问题导致失去意义
    #     # "尾无意义",
    #     # "组合无意义",#头和尾本身是合法短语，在当前关系组合下表达无意义，比如关系错配。
    # ]
    # annotation_choices = {
    #     "头短语是否合理":{
    #         "options":[
    #             "合理",
    #             "表达不通顺",
    #             "内容违背常理",
    #             "内容格式不符合要求",
    #             "和关系不匹配",
    #             "未选择"
    #         ],
    #         "desc":'<b style="color:Coral">头短语</b>是否合理？',
    #         "final_tags":[ #选项对应的最终标签,None代表不添加标签
    #             "good_head",
    #             "bad_head",
    #             "bad_head",
    #             "bad_head",
    #             "bad_head_rel",
    #             None
    #         ]
    #     },
    #     "尾短语是否合理":{
    #         "options":[
    #             "合理",
    #             "表达不通顺",
    #             "内容违背常理",
    #             "内容格式不符合要求",
    #             "和关系不匹配",
    #             "未选择"
    #         ],
    #         "desc":'<b style="color:CornflowerBlue">尾短语</b>是否合理？',
    #         "final_tags":[ #选项对应的最终标签,None代表不添加标签
    #             "good_tail",
    #             "bad_tail",
    #             "bad_tail",
    #             "bad_tail",
    #             "bad_tail_rel",
    #             None
    #         ]
    #     },
    #     "三元组整体是否成立":{
    #         "options":[
    #             "总是成立/经常成立",
    #             "有时成立/可能成立",
    #             "很难成立/从不成立/无关联",
    #             "无意义、无效表达",
    #             "未选择"
    #         ],
    #         "desc":'三元组整体是否成立？',
    #         "final_tags":[ #选项对应的最终标签,None代表不添加标签
    #             "good_triple",
    #             "good_triple",
    #             "bad_triple",
    #             "bad_triple",
    #             None
    #         ]
    #     },
    # }
    # auto_filter_mappings = [
    #     {
    #         "cond":("头短语是否合理",("表达不通顺","内容违背常理","内容格式不符合要求")), #问题，选项集合->添加到对应的黑名单
    #         "to":"head_blacklist",
    #     },
    #     {
    #         "cond":("头短语是否合理",("和关系不匹配",)),
    #         "to":"head_rel_blacklist",
    #     },
    #     {
    #         "cond":("尾短语是否合理",("表达不通顺","内容违背常理","内容格式不符合要求")),
    #         "to":"tail_blacklist",
    #     },
    #     {
    #         "cond":("尾短语是否合理",("和关系不匹配",)),
    #         "to":"tail_rel_blacklist",
    #     }
    # ]
    # enable_auto_filter = "version1"
    farthest_annotated_idx = -1
    current_idx = 0
to_be_annotated = []
with open(os.path.join(workspace,"to_be_annotated.jsonl.txt")) as f:
    for line in f:
        to_be_annotated.append(json.loads(line.rstrip()))
# with open(os.path.join(workspace,"config.json")) as f:
#     metadata = json.load(f)


In [4]:
# if enable_auto_filter == "version1":
#     #版本1,提供head、tail、head_rel、tail_rel的blacklist
#     head_blacklist = set()
#     head_rel_blacklist = set()
#     tail_blacklist = set()
#     tail_rel_blacklist = set()
#     def update_auto_filter(record):
#         for rule in auto_filter_mappings:
#             if record['ann']['choices'][rule['cond'][0]] in rule['cond'][1]:
#                 if rule['to'] == "head_blacklist":
#                     head_blacklist.add(record['triple']['h'])
#                 elif rule['to'] == 'head_rel_blacklist':
#                     head_rel_blacklist.add((record['triple']['h'],record['triple']['r']))
#                 elif rule['to'] == 'tail_blacklist':
#                     tail_blacklist.add(record['triple']['t'])
#                 elif rule['to'] == 'tail_rel_blacklist':
#                     tail_rel_blacklist.add((record['triple']['r'],record['triple']['t']))
#                 else:
#                     raise NotImplementedError
#     def check_auto_filter(item): #如果被过滤掉了，返回自动生成的记录，否则返回None
#         record = None
#         tags = []
#         if item['triple']['h'] in head_blacklist:
#             tags.append("bad_head")
#         if (item['triple']['h'],item['triple']['r']) in head_rel_blacklist:
#             tags.append("bad_head_rel")
#         if item['triple']['t'] in tail_blacklist:
#             tags.append("bad_tail")
#         if (item['triple']['r'],item['triple']['t']) in tail_rel_blacklist:
#             tags.append("bad_tail_rel")
#         if len(tags)>0: 
#             record = item.copy()
#             record['ann']={
#                 "annotator":"#auto#",
#                 "choices":None,
#                 "final_tags":tags,
#                 "time":time.strftime('%Y%m%d_%a_%H:%M:%S')
#             }
#         return record

#     if os.path.exists(os.path.join(workspace,"annotations.jsonl.txt")):    #根据标注记录，构造黑名单，用于自动过滤
#         with open(os.path.join(workspace,"annotations.jsonl.txt")) as f:
#             all_results = {}
#             for line in f:
#                 line = json.loads(line)
#                 triple = json.dumps(line['triple'],ensure_ascii=False)
#                 all_results[triple] = line
#             for result in all_results.values():
#                 if result['ann']['annotator']!="#auto#":
#                     update_auto_filter(result)
#             del all_results


In [5]:

#每次标注一个，按下按钮即跳到下一个，提供回到上一个按钮、下一个按钮、保存按钮。
#先不提供上一个、下一个按钮了，反正我们能编辑。

message_html = widgets.HTML(
    value="",
    placeholder='',
    description='',
)

data_html = widgets.HTML(
    value="",
    placeholder='Document: ',
    description='Document: ',
)



# annotation_html = widgets.HTML(
#     value = "无",
#     placeholder='既有标注：',
#     description='既有标注：'
# )

# radios = {
#     q:widgets.VBox([
#         widgets.HTML(q_args['desc']),
#         widgets.RadioButtons(
#             options = q_args['options'],
#             value = q_args['options'][-1],
#             # description=q,
#             disabled=False,
#         )
#     ])
#     for q,q_args in annotation_choices.items()
# }

ann_table = {}

submit_button = widgets.Button(description="提交并标注下一个")
# buttons = [
#     widgets.Button(description=choice)
#     for choice in annotation_choices
# ]

# button_box = widgets.GridBox(buttons)

output = widgets.Output()

def show_annotation(current_idx):
    global ann_table
    if current_idx < len(to_be_annotated):
        d = to_be_annotated[current_idx]
        # triple_html.value = f"{triple['h']} <b>{triple['r']}</b> {triple['t']}"
        # data_html.value = f'<b style="color:Coral">{triple["h"]}</b> {triple["r"]} <b style="color:CornflowerBlue">{triple["t"]}</b> <br>' + example2text(triple,name_mapping,relation_templates[triple['r']],template_arg_fn_sp)[1:]
        data_html.value = f'{d["doc"]}'
        message_html.value = f"{current_idx}/{len(to_be_annotated)}"
        #TODO 暂时不管已标注结果，后面再说
        #
        #清空标注选项，现在只清空triple标注的标注选项，硬编码 先暂时不弄了
        ann_table = {
            str(i):widgets.VBox([
                widgets.HTML(p['query'] + " <br>" + p['answer']),
                widgets.ToggleButtons(
                    options = ["0","1","2","3"],
                    value = p.get("level","0"),
                    # description=q,
                    disabled=False,
                ),
                widgets.TagsInput(
                    value=[],
                    # allowed_tags=['pizza', 'fries', 'tomatoes', 'steak'],
                    allow_duplicates=False,
                )
            ])
            for i,p in enumerate(d['generated_probes'])
        }
        
    else:
        message_html.value = '<b style="color:red">当前标注索引异常，可能已经标注完成</b>'

def save_record(record):
    global current_idx,farthest_annotated_idx
    if farthest_annotated_idx < current_idx:
        farthest_annotated_idx = current_idx
    with open(os.path.join(workspace,"annotations.jsonl.txt"),'a') as f:
        print(json.dumps(record,ensure_ascii=False),file=f)
    with open(os.path.join(workspace,"annotation_states.json"),'w') as f:
        json.dump({
            "farthest_annotated_idx":farthest_annotated_idx,
            # "annotation_choices":annotation_choices,
            # "auto_filter_mappings":auto_filter_mappings,
            # "enable_auto_filter":enable_auto_filter
        },f,ensure_ascii=False,indent=2)

def submit(choices):
    global current_idx,farthest_annotated_idx
    #TODO 加到配置里，现在是硬编码
    # if choices['头短语是否合理'] in ("合理","未选择") and choices['尾短语是否合理'] in ("合理","未选择") and choices['三元组整体是否成立'] == "未选择":
    #     message_html.value = message_html.value + '<b style="color:red">尚未做出选择！</b>'
    #     return 
    record = to_be_annotated[current_idx].copy()
    for i,p in enumerate(record['generated_probes']):
        p['level'],p['tags'] = choices[str(i)]
    # record['ann'] = {
    #     "annotator":annotator,
    #     "choices":choices,
    #     "time":time.strftime('%Y%m%d_%a_%H:%M:%S')
    # }
    #自动保存，以后可能分离成手动保存
    save_record(record)
    #更新自动过滤器
    # if enable_auto_filter:
    #     update_auto_filter(record)
    #取出下一个标注，并注意自动过滤
    while True:
        if current_idx + 1 < len(to_be_annotated):
            current_idx+=1
            # if enable_auto_filter:
            #     check_result = check_auto_filter(to_be_annotated[current_idx])
            #     if check_result is not None:
            #         save_record(check_result)
            #         continue
            show_annotation(current_idx)
        else:
            message_html.value = "<b>已经标注完成</b>"
        break
    

def on_click(b):
    with output:
        # choice_id = annotation_choices.index(b.description)
        submit({i:(ann.children[1].value,ann.children[2].value) for i,ann in ann_table.items()})


display(message_html,data_html,*ann_table.values(),submit_button,output)

show_annotation(current_idx)

submit_button.on_click(on_click)

HTML(value='', placeholder='')

HTML(value='(No triples)', description='常识知识：', placeholder='常识知识：')

Button(description='提交并标注下一个', style=ButtonStyle())

Output()

In [13]:
import ipywidgets as widgets
tags = widgets.TagsInput(
    value=[],
    # allowed_tags=['pizza', 'fries', 'tomatoes', 'steak'],
    allow_duplicates=False,
    description="Tags:"
)
submit_button = widgets.Button(description="提交并标注下一个")
display(tags,submit_button)

TagsInput(value=[], allow_duplicates=False, description='Tags:')

Button(description='提交并标注下一个', style=ButtonStyle())